In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import MiniBatchKMeans

In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import calinski_harabasz_score
import datetime as dt

In [19]:
df = pd.read_csv('./vitrina.csv')
df.head()

,party_rk,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,...,cluster_22,cluster_23,cluster_24,cluster_25,cluster_26,cluster_27,cluster_28,cluster_29,cluster_30,total_acts
0,1000083,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1000101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1000129,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10001639,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0
4,10001818,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
# Рассчитываем KMeans и считаем метрики ---> C ПОДСЧЕТОМ АКТИВНОСТИ
np.random.seed(42)

for n_clusters in range(2,10):
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, batch_size=300, n_init=n_clusters)
    cluster_labels = mbk.fit_predict(df.iloc[:, 1:-1])

    calinski_harabasz = calinski_harabasz_score(np.array(df.iloc[:, 1:-1]), cluster_labels)
    
    print(f"For n_clusters = {n_clusters}, The calinski_harabasz_score is :{calinski_harabasz}")

For n_clusters = 2, The calinski_harabasz_score is :60002.26489849452
For n_clusters = 3, The calinski_harabasz_score is :41258.515269345895
For n_clusters = 4, The calinski_harabasz_score is :34342.32009547487
For n_clusters = 5, The calinski_harabasz_score is :36632.504590656295
For n_clusters = 6, The calinski_harabasz_score is :44084.8591251859
For n_clusters = 7, The calinski_harabasz_score is :36365.35623043657
For n_clusters = 8, The calinski_harabasz_score is :36409.53396360971
For n_clusters = 9, The calinski_harabasz_score is :41774.25067422098


### Вывод -- 2 кластера - самое оптимальное разбиение выборки, гипотеза в том, что разделение идет по активности. Далее пробуем построить knn, убрав смещение по активности

In [26]:
from IPython.display import clear_output

for i in range(1,31):
    df[f'cluster_{i}'] = df[f'cluster_{i}'].apply(lambda x: 1 if x>1 else x)
    clear_output(True)
    print(f'cluster_{i}/cluster_30')

cluster_30/cluster_30


In [31]:
# Рассчитываем KMeans и считаем метрики ---> БЕЗ ПОДСЧЕТА АКТИВНОСТИ
np.random.seed(42)

for n_clusters in range(2, 11):
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, batch_size=300, n_init=n_clusters)
    cluster_labels = mbk.fit_predict(df.iloc[:, 1:-1])

    calinski_harabasz = calinski_harabasz_score(np.array(df.iloc[:, 1:-1]), cluster_labels)
    
    print(f"For n_clusters = {n_clusters}, The calinski_harabasz_score is :{calinski_harabasz}")

For n_clusters = 2, The calinski_harabasz_score is :57137.88796244441
For n_clusters = 3, The calinski_harabasz_score is :46171.60002134503
For n_clusters = 4, The calinski_harabasz_score is :45490.577077215916
For n_clusters = 5, The calinski_harabasz_score is :53789.24506583685
For n_clusters = 6, The calinski_harabasz_score is :48535.42112979007
For n_clusters = 7, The calinski_harabasz_score is :49289.36644154111
For n_clusters = 8, The calinski_harabasz_score is :47993.35420603184
For n_clusters = 9, The calinski_harabasz_score is :50252.05735498297
For n_clusters = 10, The calinski_harabasz_score is :48581.65812006601


### Вывод - 2 кластера оптимальное количество, но 5 и 9 кластеров начинают составлять бОльшую конкуренцию. Далее добавляем 2-gramm участия пользователя в 2х различных кластерах 

In [48]:
# Добавление 2gramms
for i in range(1, 31):
    for j in range(i+1, 31):
        party_in_clusters = df[(df[f'cluster_{i}']==1) & (df[f'cluster_{j}']==1)].index

        df[f'cluster_{i}_{j}'] = 0    
        df.loc[party_in_clusters, f'cluster_{i}_{j}'] = 1
    
    clear_output(True)
    print(f'i={i}')

i=30


In [5]:
# Рассчитываем KMeans и считаем метрики ---> С ДОБАВЛЕНИЕМ ДИСКРИМИНАЦИИ ПО УЧАСТИЮ В 2х РАЗЛИЧНЫХ КЛАСТЕРАХ
np.random.seed(42)

for n_clusters in range(2, 11):
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, batch_size=300, n_init=n_clusters)
    cluster_labels = mbk.fit_predict(df.iloc[:, 1:])

    calinski_harabasz = calinski_harabasz_score(np.array(df.iloc[:, 1:]), cluster_labels)
    
    print(f"For n_clusters = {n_clusters}, The calinski_harabasz_score is :{calinski_harabasz}")


For n_clusters = 2, The calinski_harabasz_score is :35538.45509601364
For n_clusters = 3, The calinski_harabasz_score is :21905.195201894643
For n_clusters = 4, The calinski_harabasz_score is :22975.84853130936
For n_clusters = 5, The calinski_harabasz_score is :26890.428473643584
For n_clusters = 6, The calinski_harabasz_score is :25456.250689615073
For n_clusters = 7, The calinski_harabasz_score is :24286.213601879892
For n_clusters = 8, The calinski_harabasz_score is :21575.666260040467
For n_clusters = 9, The calinski_harabasz_score is :22132.92865477759
For n_clusters = 10, The calinski_harabasz_score is :20485.14462068946


### Добавление дискриминации позволяет увидеть возможность разделения по 5 или 6 кластерам, но пока что разделение по 2м кластерам выигрывает

In [5]:
# Рассчитываем KMeans и считаем метрики ---> С ИСПОЛЬЗОВАНИЕМ ТОЛЬКО ДИСКРИМИНАЦИИ ПО УЧАСТИЮ В 2х РАЗЛИЧНЫХ КЛАСТЕРАХ
np.random.seed(42)

for n_clusters in range(2, 11):
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, batch_size=300, n_init=n_clusters)
    cluster_labels = mbk.fit_predict(df.iloc[:, 31:])

    calinski_harabasz = calinski_harabasz_score(np.array(df.iloc[:, 31:]), cluster_labels)
    
    print(f"For n_clusters = {n_clusters}, The calinski_harabasz_score is :{calinski_harabasz}")

For n_clusters = 2, The calinski_harabasz_score is :4245.986121402007
For n_clusters = 3, The calinski_harabasz_score is :13697.863054195925
For n_clusters = 4, The calinski_harabasz_score is :13975.944219181605
For n_clusters = 5, The calinski_harabasz_score is :9030.305419809865
For n_clusters = 6, The calinski_harabasz_score is :10143.79981481547
For n_clusters = 7, The calinski_harabasz_score is :11092.165254452448
For n_clusters = 8, The calinski_harabasz_score is :8452.325417249993
For n_clusters = 9, The calinski_harabasz_score is :5928.570915722404
For n_clusters = 10, The calinski_harabasz_score is :9336.967579988617


### Ситуация с использованием 2-gramm дала достаточно неожиданный результат, теперь лидирует разделение на 3/4 кластера и хорошее разделение можно наблюдать на 6/7 кластерах